比较Keras的fit和fit_generator的速度

In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Using TensorFlow backend.


设置hyper参数。此时20000个字的词汇数，句子长度被限制到80，批量大小为32。超参数设置如下：

In [2]:
max_features = 20000
maxlen = 80
batch_size = 32

接下来，执行数据加载和预处理。

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

建立一个模型

In [4]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

训练模型

In [5]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=1, validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 213s - loss: 0.4591 - acc: 0.7830 - val_loss: 0.3649 - val_acc: 0.8370


学习fit_generator方法

In [6]:
def batch_iter(data, labels, batch_size, shuffle=True):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1

    def data_generator():
        data_size = len(data)
        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                X, y = shuffled_data[start_index: end_index], shuffled_labels[start_index: end_index]
                yield X, y

    return num_batches_per_epoch, data_generator()

train_steps, train_batches = batch_iter(x_train, y_train, batch_size)
valid_steps, valid_batches = batch_iter(x_test, y_test, batch_size)
model.fit_generator(train_batches, train_steps, epochs=1, validation_data=valid_batches, validation_steps=valid_steps)
        

Epoch 1/1
782/782 [==============================] - 210s - loss: 0.2968 - acc: 0.8785 - val_loss: 0.4816 - val_acc: 0.7910


In [ ]:
def data_generator():
    num_batches_per_epoch = int((len(x_train) - 1) / batch_size) + 1
    data_size = len(x_train)
    while True:
        # Shuffle the data at each epoch
        if True:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = x_train[shuffle_indices]
            shuffled_labels = y_train[shuffle_indices]
        else:
            shuffled_data = x_train
            shuffled_labels = y_train

        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            X, y = shuffled_data[start_index: end_index], shuffled_labels[start_index: end_index]
            yield X, y
                
for e in range(1):
    print("epoch %d" % e)
    for X_batch, Y_batch in data_generator(): # these are chunks of ~10k pictures
        model.fit(X_batch, Y_batch, epochs=1)

epoch 0
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.2972 - acc: 0.8750
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.2009 - acc: 0.9375
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.1800 - acc: 0.9375
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.2271 - acc: 0.9062
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.1164 - acc: 1.0000
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.0813 - acc: 1.0000
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.1032 - acc: 0.9688
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.1925 - acc: 0.9375
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.0978 - acc: 0.9688
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.3538 - acc: 0.9062
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.1076 - acc: 0.9375
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.1710 - acc: 0.9